In [1]:
##########################################################################################################
# -----------------------------------------------------------------
#  mult_verify
# -----------------------------------------------------------------
#
# INPUT:
# - genus: the genus for which we want to prove a bound
# - a,b:   we assume sqrt(lambda_1-1/4) to lie in the interval [a,b]
# - param: a pair (Z,W), where:
#          * Z is a list of prescribed double roots for f(sqrt(x))
#          * W is a list of prescribed double roots for f^(sqrt(x))
#
# THE PROGRAM:
# - Computes a Fourier pair (f,f^) based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of parameters: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
# - Verifies whether the pair (f,f^) satisfies the conditions of Theorem 9.1
# - If so, gives a rigorous estimate for the bound 
#
# OUTPUT:
# - If the conditions are satisfied: the computed bound
# - If not, an error message
##########################################################################################################

def mult_verify(genus,a,b,param):
    print('\n\n----------------------------------------------------------------')
    print('mult_verify')
    print('genus: ', genus)
    print('interval: [',a,',',b,']\n\n')    
    
    (Z,W) = param
    
    # Make sure the entries of Z an W are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    a = QQ(RDF(a))
    b = QQ(RDF(b))
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double zeros for f^
    
    for elem in Z:
        if elem<=0:
            return 'The forced double zeros for u should be positive'
        
    for elem in W:
        if elem<=b-1/4:
            return 'The forced double zeros for v should be strictly larger than b'
    
    for j in range(1,m):
        if Z[j]<=Z[j-1]:
            return 'The list of forced double zeros for u is not increasing'
        
    for j in range(1,n):
        if W[j]<=W[j-1]:
            return 'The list of forced double zeros for v is not increasing'

    print('Checked the conditions on the prescribed zeroes')        
        
    (u,v) = mult_poly(a,param)
    
    u_der = u.derivative()
    v_der = v.derivative()

    u_der_der = u_der.derivative()
    v_der_der = v_der.derivative()

    print('Computed the polynomials (u,v) based on the prescribed zeroes')    
    
    if sgn(u.leading_coefficient())==1: # Check that u is eventually non-positive
        return 'f takes positive values near infinity'

    if sgn(v.leading_coefficient())==-1: # Check that v is eventually positive
        return 'f^ takes negative values near infinity'
    
    if pari.polsturm(u,[0,Infinity])>m+1:  # Check that the number of real roots of u is correct
        return 'f has an extra zero'
                
    if pari.polsturm(v,[0,Infinity])>n+1: # Check that the number of real roots of v is correct
        return 'f^ has an extra zero'
    
    # Check that the conditions at the roots are satisfied
    for j in range(m):
        if u(Z[j])!=0 :
            return 'f does not vanish at one of the prescribed zeros'  
                        
        if u_der(Z[j])!=0 :
            return 'One of the prescribed zeros of f is not double'  

        if u_der_der(Z[j])>=0 :
            return 'The second derivative of f at one of the double roots is non-negative'

    for j in range(n):
        if v(W[j])!=0 :
            return 'f^ does not vanish at one of the prescribed zeros'
                        
        if v_der(W[j])!=0 :
            return 'One of the prescribed zeros of f^ is not double'       

        if v_der_der(W[j])<=0 :
            return 'The second derivative of f^ at one of the double roots is non-positive'
    
    print('Verified the other sign conditions on f and f^')    
    
    # Make sure we can prove a lower bound on f^ on the interval [sqrt(a-1/4),sqrt(b-1/4)]
    A = a - 1/4
    B = b - 1/4    
    if pari.polsturm(2*v_der-v,[A,B])>1:    
        return 'f^ is not unimodal on the interval [sqrt(1-1/4),sqrt(b-1/4)]'
    
    def h(t):
        return v(t)*e^(-t/2)
        
    lower_bound = min(h(A),h(B))
    
    print('Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]')    
    
    # We estimate the integral term in the Selberg trace formula
    truncated_integral = CBF.integral(lambda x, _: h(x^2)*x*tanh(CBF(pi)*x),0,200)
    
    print('Computed the truncated integral')
    
    # For the remainder of the integral, we compute an indefinite integral
    # We will use that x^(2*n+1) * exp(-x^2 / 2) admits a primitive
    # of the form p_n(x)*exp(-x^2 / 2), where p_n is a polynomial
    
    # First we compute the p_n's:
    max_d = v.degree() # the maximal degree for which we need to know p_n
    p = [-1] # initializing p[0] (which is p_1)
    for n in range(1,max_d):
        p += [-x^(2*n)+2*n*p[n-1]] # adding p_n to the list, the recurrence we use is obtained
                                   # from integration by parts
    
    # Compute the primitive of x*v(x^2)*exp(-x^2/2)
    V = 0
    for n in range(max_d):
        V += v.coefficients()[n]*p[n]
        
    remainder = -V.subs(x==200)*exp(-200^2/2)

    print('Computed a bound for the remainder of the integral')
    
    integral_fhat = truncated_integral+remainder
        
    mult_bound = (RBF((2*(genus-1)*integral_fhat-h(-1/4))/lower_bound).upper()).floor()
        
    print('\nBound found: ',mult_bound)
    
    return mult_bound


##########################################################################################################
# -----------------------------------------------------------------
#  mult_poly
# -----------------------------------------------------------------
# INPUT:
# - a:     the lower end of the interval in which we assume sqrt(lambda_1-1/4) to lie
# - param: a pair (Z,W), where:
#          * Z is a set of prescribed double roots for f
#          * W is a set of prescribed double roots for f^
# 
# THE PROGRAM:
# - Computes a pair (u,v) of polynomials based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of roots: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
#
# OUTPUT:
# (u,v)
##########################################################################################################


def mult_poly(a,param):
    
    (Z,W) = param
    
    # Make sure the inputs are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    a = QQ(RDF(a))
    
    
    m = len(Z) #number of forced double roots for f
    n = len(W) #number of forced double roots for f^
    c = 2*(m+n)+2 #number of conditions imposed

    # Compile a list of Laguerre polynomials and their derivatices
    P.<x> = PolynomialRing(QQ) 
    L = [ P(gen_laguerre(n,-1/2,x)) for n in range(c)]
    L_der = [ L[n].derivative() for n in range(c)]      
    
    A = [[(-1)^j*L[j](W[k]) for j in range(c)] for k in range(n)] #for f^ to vanish at the w_k
    B = [[(-1)^j*L_der[j](W[k]) for j in range(c)] for k in range(n)] #for f^' to vanish at the w_k
    
    C = [[L[j](Z[k]) for j in range(c)] for k in range(m)] #for f to vanigh at the z_k
    D = [[L_der[j](Z[k]) for j in range(c)] for k in range(m)] #for f' to vanish at the z_k
    
    E = [[L[j](0) for j in range(c)]]
    F = [[(-1)^j*L[j](a-1/4) for j in range(c)]] 
    
    
    M = pari(matrix(QQ,A+B+C+D+E+F)) #matrix of values of the Laguerre polynomials and their derivative or combinations thereof
    Y = pari(matrix(QQ,[0]*(c-1)+[1]).transpose()) #this is the vector of target values
    X = vector(pari.matinverseimage(M,Y).sage())

    u = sum([X[j]*L[j] for j in range(c)]) #the polynomial for f 
    v = sum([(-1)^j*X[j]*L[j] for j in range(c)]) #the polynomial for f^
    
    return (u,v)


##########################################################################################################
# -----------------------------------------------------------------
#  eps
# -----------------------------------------------------------------
# INPUT:
# - g:     the genus
#
# OUTPUT:
# a number eps(g). If lamba_1(X) < 1/4+eps(g) and X has genus g, we know that m_1(X) <= 2g
##########################################################################################################
def eps(g):
    # assuming g <= 103, so that Artamoshin's bound yields the best result 
    return (pi/(4*arcsinh(sqrt((g-1)/2))))^2

In [4]:
# We verify the Table 4 for genus 4-20
# The cases of genus 2 and 3 are part of our article 
# "The Klein quartic maximizes the multiplicity of the first positive eigenvalue of the Laplacian"
# available at https://arxiv.org/abs/2111.14699

# Load the best known bounds on the lambda_1 in genus 4-20
# These are our LP bounds, except in genus 4 and genus 6, in which the best known bound is due to 
# Yang-Yau
lambda_bounds = load('list_lambda')
print('The list of upper bounds on lambda_1 we use in genus 4 to 20:',lambda_bounds)

# Load the parameters
mult_parameters = load('parameters_multiplicity')

# Verify
# Initialize the set of maximal multiplicities by setting them equal to 2*genus:
max_mult = [8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40]

# Intervals in which our bound holds
int_covered = []
for g in range(4,21):
    int_covered += [[0,RBF(0.25+eps(g)).lower()]]

for p in mult_parameters:
    genus = p[0]
    a = p[1]
    b = max(min(p[2],lambda_bounds[p[0]-4]),p[1])
    param = p[3]
    mult = mult_verify(genus,a,b,param)
    
    if mult > max_mult[p[0]-4]:
        max_mult[p[0]-4] = mult
    
    # If the intervals overlap (the parameters_multiplicity file is set up so that they do)
    # then enlarge the interval in which the bound holds
    if p[1] < int_covered[p[0]-4][1]:
        int_covered[p[0]-4][1] = b

print("\n\n")
# Check that the interval that is covered ranges up to the maximal possible lambda_1
for g in range(4,21):
    if int_covered[g-4][1] >= lambda_bounds[g-4]:
        print('The multiplicity bound in genus ',g,' holds for all surfaces')

print("\n\n")

# Print out the bounds:
print('Genus | Multiplicity bound')
for g in range(4,21):
    if g<10:
        print(g,'    | ',max_mult[g-4])
    else:
        print(g,'   | ',max_mult[g-4])


The list of upper bounds on lambda_1 we use in genus 4 to 20: [2, 1.83676600000000, 1.60000000000000, 1.48000800000000, 1.37280400000000, 1.28902400000000, 1.22218900000000, 1.16816900000000, 1.12232700000000, 1.08326000000000, 1.04921700000000, 1.01800500000000, 0.991735000000000, 0.968260000000000, 0.947180000000000, 0.928091000000000, 0.911390000000000]


----------------------------------------------------------------
mult_verify
genus:  4
interval: [ 0.828505147036686 , 1.05000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the other sign conditions on f and f^
Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  9


----------------------------------------------------------------
mult_verify
genus:  4
interval: [ 1.04850514703669 , 1.63000000000000 ]


Checked the conditions on the pres

Computed a bound for the remainder of the integral

Bound found:  20


----------------------------------------------------------------
mult_verify
genus:  10
interval: [ 0.764398551268784 , 1.22218900000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the other sign conditions on f and f^
Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  17


----------------------------------------------------------------
mult_verify
genus:  11
interval: [ 0.507590401157233 , 0.770000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the other sign conditions on f and f^
Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found

Computed the polynomials (u,v) based on the prescribed zeroes
Verified the other sign conditions on f and f^
Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  38


----------------------------------------------------------------
mult_verify
genus:  19
interval: [ 0.745542900656021 , 0.928091000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
Verified the other sign conditions on f and f^
Computed a lower bound on f^(y) for y in [sqrt(1-1/4),sqrt(b-1/4)]
Computed the truncated integral
Computed a bound for the remainder of the integral

Bound found:  23


----------------------------------------------------------------
mult_verify
genus:  20
interval: [ 0.430383651721537 , 0.755000000000000 ]


Checked the conditions on the prescribed zeroes
Computed the polynomials (u,v) based on the prescribed zeroes
